In [ ]:
import subprocess
from pathlib import Path

import geopandas
import matplotlib.pyplot as plt
import numpy
import pandas
import rasterio
import scipy.integrate
import snail.intersection
from matplotlib.colors import LogNorm
from open_gira.direct_damages import ReturnPeriodMap
from open_gira.exposure import max_vector_rasters_intersection
from open_gira.geometry import buffer_via_reprojection
from open_gira.io import read_raster_ds, write_raster_ds
from pyproj import Geod
from snail.damages import PiecewiseLinearDamageCurve

In [ ]:
base_dir = Path("../cluster")

In [ ]:
adm0 = geopandas.read_parquet(
    base_dir / "processed_data/admin-boundaries/admin-level-0.geoparquet"
).query('GID_0 == "THA"')

In [ ]:
tha_bounds = dict(adm0.bounds.iloc[0, :])
minx, miny, maxx, maxy = (
    tha_bounds["minx"],
    tha_bounds["miny"],
    tha_bounds["maxx"],
    tha_bounds["maxy"],
)
minx, miny, maxx, maxy

In [ ]:
global_raster_paths = sorted(
    Path("../results/input/jrc_flood/").glob("floodMapGL*.tif")
)
for global_tiff in global_raster_paths:
    tha_tiff = base_dir / "processed_data" / "hazard" / global_tiff.name
    if tha_tiff.exists():
        continue
    print(tha_tiff)
    subprocess.check_output(
        [
            "gdalwarp",
            "-te",
            str(minx),
            str(miny),
            str(maxx),
            str(maxy),
            str(global_tiff),
            str(tha_tiff),
        ]
    )

In [ ]:
raster_paths = sorted(
    str(p) for p in (base_dir / "processed_data" / "hazard").glob("floodMapGL*.tif")
)
rasters = pandas.DataFrame(data={"path": raster_paths})
rasters["rp"] = rasters.path.str.extract("(\d+)").astype(int)
rasters["key"] = rasters.rp.apply(lambda rp: f"jrc_flood_{rp:03d}")
rasters.sort_values(by="rp", inplace=True)
rasters

In [ ]:
# Resample to built capital raster resolution
for tha_tiff in rasters.path:
    resampled_tiff = (
        base_dir / "processed_data" / "hazard" / "3ss" / Path(tha_tiff).name
    )
    print(resampled_tiff)
    if resampled_tiff.exists():
        resampled_tiff.unlink()
        # continue
    subprocess.check_output(
        [
            "gdalwarp",
            "-t_srs",
            "EPSG:4326",
            "-te_srs",
            "EPSG:4326",
            "-te",
            "97.3745830",
            "5.6170835",
            "105.6245829",
            "20.4504168",
            "-ts",
            "9900",
            "17800",
            str(tha_tiff),
            str(resampled_tiff),
        ]
    )

In [ ]:
raster_paths_resampled = sorted(
    str(p)
    for p in (base_dir / "processed_data" / "hazard" / "3ss").glob("floodMapGL*.tif")
)
rasters_3ss = pandas.DataFrame(data={"path": raster_paths_resampled})
rasters_3ss["rp"] = rasters_3ss.path.str.extract("_rp(\d+)y").astype(int)
rasters_3ss["key"] = rasters_3ss.rp.apply(lambda rp: f"jrc_flood_{rp:03d}")
rasters_3ss.sort_values(by="rp", inplace=True)
rasters_3ss

In [ ]:
grids = set(
    snail.intersection.GridDefinition.from_raster(path) for path in rasters.path
)
assert len(grids) == 1
grid = grids.pop()
grid

In [ ]:
damage_curves = {
    "paved": PiecewiseLinearDamageCurve.from_csv(
        "../config/damage_curves/flood/road_paved.csv",
        intensity_col="inundation_depth_(m)",
        damage_col="central",
    ),
    "paved_lower": PiecewiseLinearDamageCurve.from_csv(
        "../config/damage_curves/flood/road_paved.csv",
        intensity_col="inundation_depth_(m)",
        damage_col="damage_fraction",
    ),
    "paved_upper": PiecewiseLinearDamageCurve.from_csv(
        "../config/damage_curves/flood/road_paved.csv",
        intensity_col="inundation_depth_(m)",
        damage_col="upper",
    ),
    "motorway_lower": PiecewiseLinearDamageCurve.from_csv(
        "../config/damage_curves/flood/road_motorway.csv",
        intensity_col="inundation_depth_(m)",
        damage_col="damage_fraction",
    ),
    "motorway_upper": PiecewiseLinearDamageCurve.from_csv(
        "../config/damage_curves/flood/road_motorway.csv",
        intensity_col="inundation_depth_(m)",
        damage_col="high flow",
    ),
    "rail_a": PiecewiseLinearDamageCurve.from_csv(
        "../config/damage_curves/flood/rail_railway.csv",
        intensity_col="inundation_depth_(m)",
        damage_col="damage_fraction",
    ),
    "rail_b": PiecewiseLinearDamageCurve.from_csv(
        "../config/damage_curves/flood/rail_railway.csv",
        intensity_col="inundation_depth_(m)",
        damage_col="high",
    ),
    "powerplants": PiecewiseLinearDamageCurve.from_csv(
        "../config/damage_curves/flood/powerplants.csv",
        intensity_col="depth_m",
        damage_col="damage_fraction",
    ),
}

In [ ]:
rail_edges = geopandas.read_parquet(
    base_dir
    / "transport_disruption"
    / "input"
    / "networks"
    / "rail"
    / "project-thailand"
    / "edges.gpq"
).query('from_iso_a3 == "THA" | to_iso_a3 == "THA"')
rail_edges.head(1)

In [ ]:
road_edges = geopandas.read_parquet(
    base_dir
    / "transport_disruption"
    / "input"
    / "networks"
    / "road"
    / "project-thailand"
    / "edges.gpq"
).query('from_iso_a3 == "THA" | to_iso_a3 == "THA"')
road_edges.head(1)

## Load powerplants


In [ ]:
powerplants = geopandas.read_parquet(
    base_dir / "power_flood" / "data" / "powerplants_tha.geoparquet"
)
powerplants["geom_point"] = powerplants.geometry
powerplants.geometry = buffer_via_reprojection(powerplants.geometry, 250)
powerplants.head(1)

## Powerplant RP damages


In [ ]:
powerplants = max_vector_rasters_intersection(powerplants, rasters).fillna(0)

In [ ]:
powerplants.head(2)

In [ ]:
def damage_value(
    assets_with_exposure, exposure_columns, damage_curve, cost_column, damage_cost_unit
):
    exposure = assets_with_exposure.loc[:, exposure_columns]
    damage_fraction = pandas.DataFrame(
        damage_curve.damage_fraction(exposure),
        index=exposure.index,
        columns=exposure.columns,
    )
    damage_value = damage_fraction.multiply(
        assets_with_exposure[cost_column], axis="index"
    )
    damage_value.columns = [
        f"damage_{damage_cost_unit}__{c}" for c in damage_value.columns
    ]
    return damage_value

In [ ]:
powerplants_damages = damage_value(
    powerplants, rasters.key, damage_curves["powerplants"], "construction_cost", "usd"
)

In [ ]:
class JRCFloodMap(ReturnPeriodMap):
    def __init__(self, tuple):
        # identifying string containing other, inferred attributes, should be
        # unique among any collection of maps
        self.name = tuple.key
        # name of scenario, e.g. rcp4p5, historical
        self.scenario = "historical"
        # year for which hazard map is valid (may be past, present or future)
        self.year = 2020
        # expect hazard to recur on average every return_period years
        self.return_period_years = tuple.rp

    @property
    def without_RP(self) -> str:
        return "jrc_flood"

    @property
    def without_model(self) -> str:
        return "jrc_flood"

In [ ]:
def ead(assets_with_damages, damage_cost_unit, rp_maps):
    # sort by least to most probable
    sorted_rp_maps: list[ReturnPeriodMap] = sorted(rp_maps)

    # [0, 1] valued decimal probabilities
    probabilities: list[float] = [
        rp_map.annual_probability for rp_map in sorted_rp_maps
    ]
    # family subset of grouped_direct_damages
    family_column_names: list[str] = [
        f"damage_{damage_cost_unit}__{rp_map.name}" for rp_map in sorted_rp_maps
    ]
    family_direct_damages: pandas.DataFrame = assets_with_damages[family_column_names]

    # integrate the damage as a function of probability curve using Simpson's rule
    # Simpson's rule as the function to be integrated is non-linear
    return scipy.integrate.simpson(family_direct_damages, x=probabilities, axis=1)

In [ ]:
rp_maps = [JRCFloodMap(r) for r in rasters.itertuples()]

In [ ]:
powerplants_damages["ead_usd__jrc_flood"] = ead(powerplants_damages, "usd", rp_maps)

In [ ]:
powerplants_damages.head()

In [ ]:
powerplants_with_damages = powerplants.join(powerplants_damages)

In [ ]:
for depth_col in rasters.key:
    powerplants_with_damages[f"exposure__{depth_col}"] = (
        powerplants_with_damages[depth_col] > 0
    )
    powerplants_with_damages[f"exposure_usd__{depth_col}"] = (
        powerplants_with_damages.construction_cost
        * (powerplants_with_damages[depth_col] > 0)
    )
powerplants_with_damages.filter(like="exposure__").sum()

In [ ]:
powerplants_with_damages.to_parquet(
    base_dir / "power_flood" / "powerplant_damages_jrc.gpq"
)

# Rail damages


In [ ]:
rail_costs = pandas.read_csv("../config/rehab_costs/rail.csv", comment="#").set_index(
    "asset_type"
)
rail_costs  # .loc["rail_railway", "rehab_cost_USD_per_km"]

In [ ]:
rail_edges.asset_type.unique()

In [ ]:
def line_split_exposure(edges, grid, rasters):
    exposure = snail.intersection.split_linestrings(edges.reset_index(drop=True), grid)
    exposure = snail.intersection.apply_indices(exposure, grid)
    for r in rasters.itertuples():
        with rasterio.open(r.path) as src:
            data = src.read(1)
            exposure[r.key] = snail.intersection.get_raster_values_for_splits(
                exposure, data
            )
    return exposure

In [ ]:
rail_exposure = line_split_exposure(rail_edges, grid, rasters)
rail_exposure.head(1)

In [ ]:
def line_split_damage(
    exposure,
    exposure_columns,
    costs,
    cost_col,
    damage_curves_by_asset_type,
    cost_unit="usd",
):
    geod = Geod(ellps="WGS84")
    damage_dfs = []
    for asset_type, splits_df in exposure.groupby("asset_type"):
        cost_per_km = costs.loc[asset_type, cost_col]
        splits_df["split_length_km"] = (
            splits_df.geometry.apply(geod.geometry_length) / 1e3
        )
        splits_df["rehab_cost"] = splits_df.split_length_km * cost_per_km
        damage = damage_value(
            splits_df,
            exposure_columns,
            damage_curves_by_asset_type[asset_type],
            "rehab_cost",
            cost_unit,
        )
        damage_dfs.append(splits_df.join(damage))
    return pandas.concat(damage_dfs)

In [ ]:
rail_damage_curves_a = {
    "rail_railway": damage_curves["rail_a"],
    "rail_bridge": damage_curves["rail_a"],
}
rail_damage_a = line_split_damage(
    rail_exposure,
    rasters.key,
    rail_costs,
    "rehab_cost_USD_per_km",
    rail_damage_curves_a,
)
rail_damage_a["ead_usd__jrc_flood"] = ead(rail_damage_a, "usd", rp_maps)

rail_damage_curves_b = {
    "rail_railway": damage_curves["rail_b"],
    "rail_bridge": damage_curves["rail_b"],
}
rail_damage_b = line_split_damage(
    rail_exposure,
    rasters.key,
    rail_costs,
    "rehab_cost_USD_per_km",
    rail_damage_curves_b,
)
rail_damage_b["ead_usd__jrc_flood"] = ead(rail_damage_b, "usd", rp_maps)

In [ ]:
for depth_col in rasters.key:
    rail_damage_a[f"exposure_km__{depth_col}"] = rail_damage_a.split_length_km * (
        rail_damage_a[depth_col] > 0
    )
rail_damage_a.filter(like="exposure_km__jrc_flood_").sum()

# Road damages


In [ ]:
road_costs = pandas.read_csv("../config/rehab_costs/road.csv", comment="#").set_index(
    "asset_type"
)
road_costs

In [ ]:
road_edges.asset_type.unique()

In [ ]:
road_damage_curves_lower = {
    "road_trunk": damage_curves["motorway_lower"],
    "road_motorway": damage_curves["motorway_lower"],
    "road_primary": damage_curves["paved_lower"],
    "road_secondary": damage_curves["paved_lower"],
    "road_bridge": damage_curves["paved_lower"],
}
road_damage_curves_upper = {
    "road_trunk": damage_curves["motorway_upper"],
    "road_motorway": damage_curves["motorway_upper"],
    "road_primary": damage_curves["paved_upper"],
    "road_secondary": damage_curves["paved_upper"],
    "road_bridge": damage_curves["paved_upper"],
}

In [ ]:
road_exposure = line_split_exposure(road_edges, grid, rasters)

In [ ]:
road_damage_lower = line_split_damage(
    road_exposure,
    rasters.key,
    road_costs,
    "rehab_cost_USD_per_km_per_lane",
    road_damage_curves_lower,
)
road_damage_upper = line_split_damage(
    road_exposure,
    rasters.key,
    road_costs,
    "rehab_cost_USD_per_km_per_lane",
    road_damage_curves_upper,
)

In [ ]:
road_damage_upper["ead_usd__jrc_flood"] = ead(road_damage_upper, "usd", rp_maps)
road_damage_lower["ead_usd__jrc_flood"] = ead(road_damage_lower, "usd", rp_maps)

In [ ]:
for depth_col in rasters.key:
    road_damage_upper[f"exposure_km__{depth_col}"] = (
        road_damage_upper.split_length_km * (road_damage_upper[depth_col] > 0)
    )
road_damage_upper.filter(like="exposure_km__jrc_flood_").sum()

# Plotting

- exposure of infrastructure assets (numbers of power plants, km of roads) in Thailand in floods of varying return periods
- total direct damages to infrastructure assets (power plants, roads) in Thailand in floods of varying return periods
- total direct damages to buildings in Thailand in floods of varying return periods
- transport infrastructure exposure in 1: and 1: return period floods, illustrating where most trade flows are exposed


In [ ]:
# Explore styles
for s in plt.style.available:
    print(s)
plt.style.use("seaborn-v0_8-muted")

In [ ]:
# Aggregate exposure for each return period
total_powerplants_exposure = (
    powerplants_with_damages.filter(like="exposure__jrc_flood_").sum().astype(int)
)

# Extract return periods from column names
return_periods = [int(col.split("_")[-1]) for col in total_powerplants_exposure.index]
probabilities = [1 / rp for rp in return_periods]

# Plot the exposure
fig = plt.figure(figsize=(10, 6), facecolor="white", layout="constrained")
fig.patch.set_facecolor("white")
plt.plot(probabilities, total_powerplants_exposure, marker=".")
plt.xlabel("Return Period (years)")
plt.ylabel("Total Powerplant Exposure (sites)")
plt.title(
    """Total exposure of powerplants in Thailand
to river floods of varying return periods"""
)

plt.xticks(probabilities, return_periods, rotation=90)
plt.yticks(range(50, 55))
plt.savefig(base_dir / "figures" / "rp_exposure_jrc-flood_power-sites.png")
None

In [ ]:
# Aggregate exposure for each return period
total_power_damage = (
    powerplants_with_damages.filter(like="damage_usd__jrc_flood_").sum() * 1e-9
)

# Extract return periods from column names
return_periods = [int(col.split("_")[-1]) for col in total_power_damage.index]
probabilities = [1 / rp for rp in return_periods]

# Plot the exposure
fig = plt.figure(figsize=(10, 6), facecolor="white", layout="constrained")
fig.patch.set_facecolor("white")
plt.plot(probabilities, total_power_damage, marker=".")
plt.xlabel("Return Period (years)")
plt.ylabel("Total Powerplant Damage (billion USD)")
plt.title(
    """Total direct damages to powerplants in Thailand
for river floods of varying return periods"""
)

plt.xticks(probabilities, return_periods, rotation=90)
plt.savefig(base_dir / "figures" / "rp_damage_jrc-flood_power.png")
None

In [ ]:
# Aggregate exposure for each return period
total_rail_exposure = rail_damage_a.filter(like="exposure_km__jrc_flood_").sum() * 1e-3

# Extract return periods from column names
return_periods = [int(col.split("_")[-1]) for col in total_rail_exposure.index]
probabilities = [1 / rp for rp in return_periods]

# Plot the exposure
fig = plt.figure(figsize=(10, 6), facecolor="white", layout="constrained")
fig.patch.set_facecolor("white")
plt.plot(probabilities, total_rail_exposure, marker=".")
plt.xlabel("Return Period (years)")
plt.ylabel("Total Railway Exposure ('000 km)")
plt.title(
    """Total exposure of railways in Thailand
to river floods of varying return periods"""
)

plt.xticks(probabilities, return_periods, rotation=90)
plt.savefig(base_dir / "figures" / "rp_exposure_jrc-flood_rail.png")
None

In [ ]:
# Aggregate total damages for each return period
total_rail_damages_a = rail_damage_a.filter(like="damage_usd__jrc_flood_").sum() * 10e-9
total_rail_damages_b = rail_damage_b.filter(like="damage_usd__jrc_flood_").sum() * 10e-9
total_rail_damages_center = (total_rail_damages_a + total_rail_damages_b) / 2

# Extract return periods from column names
return_periods = [int(col.split("_")[-1]) for col in total_rail_damages_a.index]
probabilities = [1 / rp for rp in return_periods]

# Plot the damages
fig = plt.figure(figsize=(10, 6), facecolor="white", layout="constrained")
fig.patch.set_facecolor("white")
plt.plot(probabilities, total_rail_damages_center, marker=".")
plt.fill_between(probabilities, total_rail_damages_a, total_rail_damages_b, alpha=0.5)
plt.xlabel("Return Period (years)")
plt.ylabel("Total Rail Damages (billion USD)")
plt.title(
    """Total direct damages to railways in Thailand
for river floods of varying return periods"""
)

plt.xticks(probabilities, return_periods, rotation=90)
plt.savefig(base_dir / "figures" / "rp_damage_jrc-flood_rail.png")
None

In [ ]:
# Aggregate exposure for each return period
total_road_exposure = (
    road_damage_upper.filter(like="exposure_km__jrc_flood_").sum() * 1e-3
)

# Extract return periods from column names
return_periods = [int(col.split("_")[-1]) for col in total_road_exposure.index]
probabilities = [1 / rp for rp in return_periods]

# Plot the exposure
fig = plt.figure(figsize=(10, 6), facecolor="white", layout="constrained")
fig.patch.set_facecolor("white")
plt.plot(probabilities, total_road_exposure, marker=".")
plt.xlabel("Return Period (years)")
plt.ylabel("Total road Exposure ('000 km)")
plt.title(
    """Total exposure of roads in Thailand
to river floods of varying return periods"""
)

plt.xticks(probabilities, return_periods, rotation=90)
plt.savefig(base_dir / "figures" / "rp_exposure_jrc-flood_road.png")
None

In [ ]:
# Aggregate total damages for each return period
dmg_road_a = road_damage_lower.filter(like="damage_usd__jrc_flood_").sum() * 10e-9
dmg_road_b = road_damage_upper.filter(like="damage_usd__jrc_flood_").sum() * 10e-9
dmg_road_c = (dmg_road_a + dmg_road_b) / 2

# Extract return periods from column names
return_periods = [int(col.split("_")[-1]) for col in dmg_road_a.index]
probabilities = [1 / rp for rp in return_periods]

# Plot the damages
fig = plt.figure(figsize=(10, 6), facecolor="white", layout="constrained")
fig.patch.set_facecolor("white")
plt.plot(probabilities, dmg_road_c, marker=".")
plt.fill_between(probabilities, dmg_road_a, dmg_road_b, alpha=0.5)
plt.xlabel("Return Period (years)")
plt.ylabel("Total Rail Damages (billion USD)")
plt.title(
    """Total direct damages to roads in Thailand
for river floods of varying return periods"""
)

plt.xticks(probabilities, return_periods, rotation=90)
plt.savefig(base_dir / "figures" / "rp_damage_jrc-flood_road.png")
None

In [ ]:
# Aggregate exposure for each return period
exposure_by_asset_type = {}
for asset_type, asset_exposure in road_damage_upper.groupby("asset_type"):
    exposure_by_asset_type[asset_type] = (
        asset_exposure.filter(like="exposure_km__jrc_flood_").sum() * 10e-3
    )

In [ ]:
fig, ax = plt.subplots(facecolor="white", layout="constrained")
fig.patch.set_facecolor("white")

# Stacked area
# ax.stackplot(probabilities, damage_by_asset_type.values(),
#              labels=damage_by_asset_type.keys(), alpha=0.8)

# Grouped bar
x = numpy.arange(len(return_periods))
width = 1 / (len(exposure_by_asset_type) + 1)  # the width of the bars
multiplier = 0
for attribute, measurement in exposure_by_asset_type.items():
    offset = width * multiplier
    rects = ax.bar(
        x + offset, measurement, width, label=attribute.replace("road_", "").title()
    )
    multiplier += 1
ax.set_ylim(0, 800)
plt.xlabel("Return Period (years)")
plt.ylabel("Sector Road Exposure ('000 km)")
plt.title(
    """Direct Exposure of roads in Thailand
to river floods of varying return periods"""
)
plt.legend(facecolor="white", framealpha=1)

# Stacked area
# plt.xticks(probabilities, return_periods, rotation=90)

# Grouped bar
plt.xticks(x + width * 2, return_periods)

plt.savefig(base_dir / "figures" / "rp_exposure_jrc-flood_road_sector.png")

In [ ]:
damage_by_asset_type = {}
for asset_type, asset_damage in road_damage_upper.groupby("asset_type"):
    damage_by_asset_type[asset_type] = (
        asset_damage.filter(like="damage_usd__jrc_flood_").sum() * 10e-9
    )

In [ ]:
fig, ax = plt.subplots(facecolor="white", layout="constrained")
fig.patch.set_facecolor("white")

# Stacked area
# ax.stackplot(probabilities, damage_by_asset_type.values(),
#              labels=damage_by_asset_type.keys(), alpha=0.8)

# Grouped bar
x = numpy.arange(len(return_periods))
width = 1 / (len(damage_by_asset_type) + 1)  # the width of the bars
multiplier = 0
for attribute, measurement in damage_by_asset_type.items():
    offset = width * multiplier
    rects = ax.bar(
        x + offset, measurement, width, label=attribute.replace("road_", "").title()
    )
    multiplier += 1
ax.set_ylim(0, 1400)
plt.xlabel("Return Period (years)")
plt.ylabel("Sector Road Damages (billion USD)")
plt.title(
    """Direct damages to roads in Thailand
for river floods of varying return periods"""
)
plt.legend(facecolor="white", framealpha=1, loc="upper left")

# Stacked area
# plt.xticks(probabilities, return_periods, rotation=90)

# Grouped bar
plt.xticks(x + width * 2, return_periods)

plt.savefig(base_dir / "figures" / "rp_damage_jrc-flood_road_sector.png")

In [ ]:
fig, ax1 = plt.subplots(facecolor="white", layout="constrained")
fig.patch.set_facecolor("white")

ax2 = ax1.twinx()

ax1.set_title(
    """Total exposure of infrastructure in Thailand
to river floods of varying return periods"""
)

ax1.set_xticks(probabilities, return_periods, rotation=90)
ax1.set_xlabel("Return Period (years)")

ax1.set_ylabel("Total Road and Rail Exposure ('000 km)")
ax1.set_ylim(0, 120)

line_power = ax2.plot(
    probabilities,
    total_powerplants_exposure,
    marker=".",
    color="red",
    zorder=2.5,
    label="Power",
)
line_rail = ax1.plot(
    probabilities, total_rail_exposure, marker=".", zorder=2.5, label="Rail"
)
line_road = ax1.plot(
    probabilities, total_road_exposure, marker=".", zorder=2.5, label="Road"
)


ax2.set_ylabel("Total Powerplant Exposure (sites)")
ax2.set_ylim(30, 60)

ax2.grid(False)

fig.legend(
    handles=[line_road[0], line_power[0], line_rail[0]],
    loc="center right",
    bbox_to_anchor=(0.9, 0.4),
)
# fig.tight_layout()
plt.savefig(base_dir / "figures" / "rp_exposure_jrc-flood_all.png")
None

In [ ]:
fig, ax = plt.subplots()
fig.patch.set_facecolor("white")

ax.set_title(
    """Total direct damages to infrastructure in Thailand
for river floods of varying return periods"""
)

ax.set_xticks(probabilities, return_periods, rotation=90)
ax.set_xlabel("Return Period (years)")

ax.set_ylabel("Total Damage (billion USD)")
# ax.set_yscale('log')

line_power = ax.plot(
    probabilities,
    total_power_damage,
    marker=".",
    color="red",
    zorder=2.5,
    label="Power",
)
line_rail = ax.plot(
    probabilities, total_rail_damages_center, marker=".", zorder=2.5, label="Rail"
)
line_road = ax.plot(probabilities, dmg_road_c, marker=".", zorder=2.5, label="Road")


fig.legend(
    handles=[line_road[0], line_rail[0], line_power[0]],
    loc="center right",
    bbox_to_anchor=(0.9, 0.2),
)
fig.tight_layout()
plt.savefig(base_dir / "figures" / "rp_damage_jrc-flood_all.png")
None

# Trade flow measures


In [ ]:
list(
    (base_dir / "transport_disruption" / "flow_allocation" / "project-thailand").glob(
        "*pq"
    )
)

In [ ]:
flow_edges = geopandas.read_parquet(
    base_dir
    / "transport_disruption"
    / "flow_allocation"
    / "project-thailand"
    / "edges.gpq"
)

In [ ]:
def join_id(row):
    ids = sorted([row.to_id, row.from_id])
    return f"{ids[0]}--{ids[1]}"


road_flow_edges = flow_edges.query("mode == 'road'").copy()
road_flow_edges["join_id"] = road_flow_edges.apply(join_id, axis=1)
road_flow_edges.shape

In [ ]:
delta = geopandas.read_parquet(
    base_dir / "transport_disruption" / "thailand_floods_trade_flow_delta.gpq"
)

In [ ]:
delta["join_id"] = delta.apply(join_id, axis=1)
delta.head(2)

In [ ]:
road_damage_upper.columns

In [ ]:
road_damage_to_group = road_damage_upper[
    [
        "from_id",
        "to_id",
        "jrc_flood_010",
        "jrc_flood_020",
        "jrc_flood_050",
        "jrc_flood_100",
        "jrc_flood_200",
        "jrc_flood_500",
        "damage_usd__jrc_flood_010",
        "damage_usd__jrc_flood_020",
        "damage_usd__jrc_flood_050",
        "damage_usd__jrc_flood_100",
        "damage_usd__jrc_flood_200",
        "damage_usd__jrc_flood_500",
    ]
].copy()
road_damage_to_group.from_id = road_damage_to_group.from_id.apply(
    lambda id: f"road_{id}"
)
road_damage_to_group.to_id = road_damage_to_group.to_id.apply(lambda id: f"road_{id}")
road_damage_to_group["join_id"] = road_damage_to_group.apply(join_id, axis=1)

road_damage_grouped = (
    road_damage_to_group.drop(columns=["from_id", "to_id"])
    .groupby(
        [
            "join_id",
        ]
    )
    .sum()
    .reset_index()
    .set_index("join_id")
)

In [ ]:
flows_and_damages = delta.set_index("join_id").join(road_damage_grouped)
flows_and_damages.ref.fillna("-", inplace=True)
flows_and_damages.fillna(0, inplace=True)

In [ ]:
flows_and_damages.head()

In [ ]:
for key in rasters.key:
    # damage_mask = (flows_and_damages[f"damage_usd__{key}"] > 0)  # alternative assume disrupted if nonzero damage
    depth_mask = flows_and_damages[key] > 0.5  # assume disrupted if flood depth > 0.5m
    flows_and_damages[f"flow_kusd__{key}"] = numpy.maximum(
        flows_and_damages.nominal_value_kusd * depth_mask,
        0,
    )
    flows_and_damages[f"flow_td-1__{key}"] = numpy.maximum(
        flows_and_damages.nominal_volume_tons * depth_mask,
        0,
    )

In [ ]:
def plot_in_country_trade_flows(
    ax,
    edges: geopandas.GeoDataFrame,
    boundaries: geopandas.GeoDataFrame,
    title: str,
    column: str,
    column_label: str,
    vmin: float,
    vmax: float,
    legend: bool,
) -> None:
    to_plot = edges[edges[column] != 0]
    to_plot_zeros = edges[edges[column] <= 0]
    norm = LogNorm(10 ** numpy.log10(vmin), 10 ** numpy.log10(vmax))
    to_plot_zeros.plot(
        color="#eeeeee",
        ax=ax,
        linewidth=0.2,
    )
    to_plot.sort_values(column).plot(
        column,
        ax=ax,
        legend=legend,
        norm=norm,
        alpha=0.7,
        legend_kwds={"shrink": 0.5, "label": column_label},
        cmap="magma_r",
    )
    xmin, xmax = ax.get_xlim()
    ymin, ymax = ax.get_ylim()
    ax.set_yticks(range(5, 22))
    xticks = range(98, 106)
    ax.set_xticks(xticks, labels=[str(t) for t in xticks])
    boundaries.plot(ax=ax, lw=0.5, alpha=0.2)
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)
    ax.grid(alpha=0.3)
    ax.set_xlabel("Longitude [deg]")
    ax.set_ylabel("Latitude [deg]")
    ax.set_title(title)
    return ax


f, axs = plt.subplots(1, 2, figsize=(15, 12))

vmin = flows_and_damages.volume_tons[flows_and_damages.volume_tons > 0].quantile(0.1)
vmax = flows_and_damages.volume_tons.max()
plot_in_country_trade_flows(
    axs[0],
    flows_and_damages,
    adm0,
    "10y return period",
    "flow_td-1__jrc_flood_010",
    "Flow volume [t/d]",
    vmin,
    vmax,
    False,
)
plot_in_country_trade_flows(
    axs[1],
    flows_and_damages,
    adm0,
    "100y return period",
    "flow_td-1__jrc_flood_100",
    "Flow volume [t/d]",
    vmin,
    vmax,
    True,
)
f.tight_layout()
f.savefig(base_dir / "figures" / "rp_transport_disruption_010-100.png")

In [ ]:
flows_and_damages_with_ref = flows_and_damages  # .join(delta.set_index("join_id")[["name_en", "ref", "osm_way_id"]])
flows_and_damages_with_ref.osm_way_id = flows_and_damages_with_ref.osm_way_id.fillna(
    -1
).astype("int")
flows_and_damages_with_ref.ref = flows_and_damages_with_ref.ref.fillna("-").astype(
    "str"
)

In [ ]:
vol_disruption = (
    flows_and_damages_with_ref[
        [
            "name_en",
            "ref",
            "tag_highway",
            "osm_way_id",
            "flow_td-1__jrc_flood_010",
            "flow_td-1__jrc_flood_100",
            "damage_usd__jrc_flood_010",
            "damage_usd__jrc_flood_100",
        ]
    ].sort_values("flow_td-1__jrc_flood_100", ascending=False)
    # .head(200)
)
vol_disruption.set_index("osm_way_id").loc[1287965816, :]

In [ ]:
flows_and_damages_with_ref[
    flows_and_damages_with_ref["flow_td-1__jrc_flood_010"]
    < flows_and_damages_with_ref["flow_td-1__jrc_flood_500"]
].shape

In [ ]:
vol_disruption = (
    flows_and_damages_with_ref[
        [
            "name_en",
            "ref",
            "tag_highway",
            "osm_way_id",
            "flow_td-1__jrc_flood_010",
            "flow_td-1__jrc_flood_100",
            "damage_usd__jrc_flood_010",
            "damage_usd__jrc_flood_100",
        ]
    ]
    .groupby(["name_en"])
    .agg(
        {
            "ref": "first",
            "osm_way_id": "first",
            "tag_highway": "first",
            "flow_td-1__jrc_flood_010": "max",
            "flow_td-1__jrc_flood_100": "max",
            "damage_usd__jrc_flood_010": "sum",
            "damage_usd__jrc_flood_100": "sum",
        }
    )
    .sort_values("flow_td-1__jrc_flood_100", ascending=False)
    .reset_index()
    # .drop_duplicates(subset="name_en")
    .head(20)[
        [
            "osm_way_id",
            "flow_td-1__jrc_flood_010",
            "damage_usd__jrc_flood_010",
            "flow_td-1__jrc_flood_100",
            "damage_usd__jrc_flood_100",
            "name_en",
            "ref",
            "tag_highway",
        ]
    ]
)

for col in ["flow_td-1__jrc_flood_010", "flow_td-1__jrc_flood_100"]:
    vol_disruption[col] = vol_disruption[col].round(-1).astype(int)
for col in ["damage_usd__jrc_flood_010", "damage_usd__jrc_flood_100"]:
    vol_disruption[col] = (vol_disruption[col] * 1e-3).round(-1).astype(int)


vol_disruption.to_csv(base_dir / "figures" / "rp_transport_disruption_010-100.csv")
vol_disruption

# Buildings exposure and damage calculations


In [ ]:
# Input value in terms of built capital from GIRI BEM
# rr_val_5x5_tif = base_dir / "processed_data/giri-bem/bem_5x5_valfis_res__THA.tif"
# nr_val_5x5_tif = base_dir / "processed_data/giri-bem/bem_5x5_valfis_nres__THA.tif"

# Input volume in terms of building volume from JRC GHSL
# rr_vol_3ss_tif = base_dir / "processed_data/ghsl/ghs_built_v_res_3ss__THA.tif"
# nr_vol_3ss_tif = base_dir / "processed_data/ghsl/ghs_built_v_nres_3ss__THA.tif"

# Processed into value in terms of built capital, downscaled on volume
# see notebooks/built-capital-gva.ipynb and scripts/building_damages.py
rr_val_3ss_tif = base_dir / "processed_data/giri-bem/bem_3ss_valfis_nres__THA.tif"
nr_val_3ss_tif = base_dir / "processed_data/giri-bem/bem_3ss_valfis_nres__THA.tif"

In [ ]:
rr_val, rr_ds = read_raster_ds(rr_val_3ss_tif)
nr_val, nr_ds = read_raster_ds(nr_val_3ss_tif)

In [ ]:
rr_damage_curve = PiecewiseLinearDamageCurve.from_csv(
    "../config/damage_curves/flood/residential_asia.csv",
    intensity_col="inundation_depth_(m)",
    damage_col="damage_fraction",
)
nr_damage_curve = PiecewiseLinearDamageCurve.from_csv(
    "../config/damage_curves/flood/commercial_asia.csv",
    intensity_col="inundation_depth_(m)",
    damage_col="damage_fraction",
)

In [ ]:
list(rasters_3ss.path)

In [ ]:
def calculate_damage_val(depth, value, damage_curve):
    damage_fraction = damage_curve.damage_fraction(depth)
    damage_value = value * damage_fraction
    return damage_value

In [ ]:
rr_rp = {}
nr_rp = {}
for row in rasters_3ss.itertuples():
    print(row)
    rp_depth, rp_depth_ds = read_raster_ds(row.path)
    transform = rp_depth_ds.transform
    rr_dmg = calculate_damage_val(rp_depth, rr_val, rr_damage_curve)
    nr_dmg = calculate_damage_val(rp_depth, nr_val, nr_damage_curve)
    write_raster_ds(
        base_dir / "buildings_flood" / f"dmg_val_res__{row.key}.tif", rr_dmg, transform
    )
    write_raster_ds(
        base_dir / "buildings_flood" / f"dmg_val_nres__{row.key}.tif", nr_dmg, transform
    )
    rr_rp[row.rp] = rr_dmg
    nr_rp[row.rp] = nr_dmg

In [ ]:
rps = []
rr = []
nr = []
for rp, rr_dmg in rr_rp.items():
    rps.append(rp)
    rr.append(rr_dmg.sum() * 1e-9)
    nr.append(nr_rp[rp].sum() * 1e-9)

In [ ]:
fig, ax = plt.subplots(facecolor="white", layout="constrained")
fig.patch.set_facecolor("white")

ax.set_title(
    """Total direct damages to built capital in Thailand
for river floods of varying return periods"""
)
probabilities = [1 / rp for rp in rps]
ax.set_xticks(probabilities, rps, rotation=90)
ax.set_xlabel("Return Period (years)")

ax.set_ylabel("Total Damage (billion USD)")
# ax.set_yscale('log')

line_res = ax.plot(probabilities, rr, marker=".", zorder=2.5, label="Residential")
line_nres = ax.plot(probabilities, nr, marker=".", zorder=2.5, label="Non-residential")


fig.legend(
    handles=[line_nres[0], line_res[0]],
    loc="center right",
    bbox_to_anchor=(0.9, 0.75),
)
plt.savefig(base_dir / "figures" / "rp_damage_jrc-flood_buildings.png")
None